In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

from matplotlib import rc
rc('font',family='AppleGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
from sklearn import linear_model
from scipy import stats
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import glob
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib.request
import ssl
import requests
from urllib.request import urlopen
from selenium import webdriver
import time
import re


In [2]:
### 각 연도별 데이터에서 필요 없는 행 열 제거 후 저장

def preprocessing(*yeardatas):
    for yeardata in yeardatas:
        data = pd.read_csv('/Users/hanjaemin/Desktop/data/KOBIS_연도별박스오피스_'+yeardata+'년도.csv')
        data = data.drop(index=[0, 1, 2], axis=0)
        data = data.rename(columns=data.iloc[0])
        data = data.drop(data.index[0])
        data = data.drop(columns=['순위'], axis=1)
        data = data.drop(columns=['매출액 점유율'], axis=1)
        data = data.drop(columns=['대표국적'], axis=1)
        data = data.reset_index(drop=True)
        data = data.drop(index=[len(data)-1], axis=0)
        data = data.head(100)
        data.to_csv('/Users/hanjaemin/Desktop/preprocessing/'+yeardata+'.csv',index=False)


In [3]:
preprocessing('2010', '2011', '2012', '2013','2014', '2015', '2016', '2017', '2018', '2019', '2020')

In [ ]:
### 각 csv파일 합치기

In [4]:

input_data = '/Users/hanjaemin/Desktop/preprocessing'
output_data = '/Users/hanjaemin/Desktop/preprocessing/2010~2020.csv'

data_list = glob.glob(os.path.join(input_data, '20*'))
all_data=[]
for file in data_list:
    df = pd.read_csv(file) 
    all_data.append(df)

combinedata = pd.concat(all_data, axis=0, ignore_index=True)
combinedata = combinedata.drop_duplicates(['영화명'])
combinedata['매출액']=combinedata['매출액'].str.replace(',','').astype('float')
combinedata = combinedata.sort_values(by=['매출액'],axis=0,ascending=False)
combinedata.reset_index(drop=True,inplace=True)

In [5]:
combinedata.to_csv(output_data,index=False)
combinedata.to_excel('output_data.xlsx',index=False)

In [6]:
combinedata = pd.read_csv('/Users/hanjaemin/Desktop/preprocessing/2010~2020.csv')

In [7]:
combinedata

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,국적,배급사
0,극한직업,2019.1.23,1.396518e+11,"16,265,618","2,003","292,584",한국,(주)씨제이이엔엠
1,명량,2014.7.30,1.357484e+11,"17,613,682","1,587","188,611",한국,(주)씨제이이엔엠
2,어벤져스: 엔드게임,2019.4.24,1.221827e+11,"13,934,592","2,835","242,001",미국,월트디즈니컴퍼니코리아 유한책임회사
3,베테랑,2015.8.5,1.051682e+11,"13,414,009","1,115","199,231",한국,(주)씨제이이엔엠
4,신과함께-인과 연,2018.8.1,1.026661e+11,"12,274,996","2,235","179,981",한국,롯데컬처웍스(주)롯데엔터테인먼트
...,...,...,...,...,...,...,...,...
1042,더 프린세스 : 도둑맞은 공주,2020.8.12,4.574497e+08,"55,828",423,"4,192",우크라이나,(주)팝엔터테인먼트
1043,프리키 데스데이,2020.11.25,4.512257e+08,"51,812",631,"12,882",미국,유니버설픽쳐스인터내셔널 코리아(유)
1044,날씨의 아이,2019.10.30,4.449562e+08,"58,968",194,"5,921",일본,"메가박스중앙(주)플러스엠,(주)미디어캐슬,워터홀컴퍼니(주)"
1045,눈의 여왕4,2019.12.24,4.446496e+08,"67,159",271,"2,239",러시아,(주)넥스트엔터테인먼트월드(NEW)


In [8]:
movie_list=combinedata['영화명']
movie_list

0                   극한직업
1                     명량
2             어벤져스: 엔드게임
3                    베테랑
4              신과함께-인과 연
              ...       
1042    더 프린세스 : 도둑맞은 공주
1043            프리키 데스데이
1044              날씨의 아이
1045              눈의 여왕4
1046              다크 나이트
Name: 영화명, Length: 1047, dtype: object

In [21]:
### 영화 이름, 장르 , 감독, 주연배우, 영화등급 crawling 

name_list = []
genre_list = []
director_list = []
actor_list = []
film_rank_list = []


In [22]:
## 특정 url로 request하거나 urlopen시 아래와 같은 에러가 발생하는 경우해결
ssl._create_default_https_context = ssl._create_unverified_context

driver = webdriver.Chrome('/Users/hanjaemin/PycharmProjects/acorn/chromedriver')
driver.get("https://movie.naver.com/")

In [23]:
for movie in movie_list:
    elem = driver.find_element_by_id("ipt_tx_srch")
    elem.send_keys(movie)
    time.sleep(1)
    try:
        path = driver.find_element_by_xpath('//*[@id="jAutoMV"]/ul/li[1]/a')
        path.click()
        
    except:
        elem.send_keys(Keys.RETURN)
        driver.find_element_by_xpath('//*[@id="old_content"]/ul[2]/li[1]/dl/dt/a').click()

    name = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > h3 > a')

    genre = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(1) > a:nth-child(1)')

    actor = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(6) > p')

    director = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(4) > p > a')

    film_rank = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(8) > p')


    if len(name) == 0 :
        name_list.append('')
    else:
        for i in range(len(name)):
            name_list.append(name[i].text)


    if len(genre) == 0 :
        genre_list.append('')
    else:
        for i in range(len(genre)):
            genre_list.append(genre[i].text)


    if len(actor) == 0 :
        actor_list.append('')
    else:
        for i in range(len(actor)):
            actor_list.append(actor[i].text)


    if len(director) == 0 :
        director_list.append('')
    else:
        for i in range(len(director)):
            if i ==1:
                break
            director_list.append(director[i].text)


    if len(film_rank) == 0 :
        film_rank_list.append('')
    else:
        for i in range(len(film_rank)):
            film_rank_list.append(film_rank[i].text)










In [24]:
len(film_rank_list)

1047

In [41]:
len(name_list)

1047

In [26]:
len(genre_list)

1047

In [27]:
len(director_list)

1047

In [28]:
len(genre_list)

1047

In [29]:
len(actor_list)

1047

In [49]:
data={'director':director_list,
      'genre':genre_list,
      'actor':actor_list,
      'filmrank':film_rank_list}
crawling_data=pd.DataFrame(data)

In [50]:
crawling_data

,director,genre,actor,filmrank
0,이병헌,코미디,"류승룡(고반장), 이하늬(장형사), 진선규(마형사)",[국내] 15세 관람가
1,정세교,다큐멘터리,"김한민, 오타니 료헤이, 이해영",[국내] 12세 관람가
2,안소니 루소,액션,"로버트 다우니 주니어(토니 스타크 / 아이언맨), 크리스 에반스(스티브 로저스 / ...",[국내] 12세 관람가
3,류승완,액션,"황정민(서도철), 유아인(조태오), 유해진(최상무)",[국내] 15세 관람가
4,김용화,판타지,"하정우(강림), 주지훈(해원맥), 김향기(덕춘)",[국내] 12세 관람가
...,...,...,...,...
1042,올레 말라므,애니메이션,"사문영(밀라 목소리), 남도형(루슬란 목소리), 서반석(레스터 목소리)",[국내] 전체 관람가
1043,크리스토퍼 랜던,코미디,"빈스 본, 캐서린 뉴튼",[국내] 15세 관람가
1044,신카이 마코토,애니메이션,"심규혁(호다카), 김유림(히나), 최한(스가)",[국내] 15세 관람가
1045,로버트 렌스,애니메이션,"양정화(눈의 여왕 한국어 목소리), 박지윤(겔다 한국어 목소리), 임채헌(헤럴드 한...",[국내] 전체 관람가


In [51]:
crawling_data.to_csv('crawling_data.csv',index=False)
crawling_data.to_excel('crawling_data.xlsx',index=False)

In [52]:
final_data = pd.concat([combinedata, crawling_data], axis=1)

In [53]:
final_data.rename(columns={'영화명': 'moviename',
                          '개봉일' : 'openingdate',
                          '매출액' : 'sales',
                          '관객수' : 'attendance',
                          '스크린수' : 'screen',
                          '상영횟수' : 'playcount',
                          '국적' : 'country',
                          '배급사' : 'distributor'},inplace=True)

In [61]:
final_data=pd.read_csv('/Users/hanjaemin/PycharmProjects/Project/final_data.csv')

In [62]:
final_data

,moviename,openingdate,sales,attendance,screen,playcount,country,distributor,director,genre,actor,filmrank
0,극한직업,2019.1.23,1.396518e+11,"16,265,618","2,003","292,584",한국,(주)씨제이이엔엠,이병헌,코미디,"류승룡(고반장), 이하늬(장형사), 진선규(마형사)",[국내] 15세 관람가
1,명량,2014.7.30,1.357484e+11,"17,613,682","1,587","188,611",한국,(주)씨제이이엔엠,정세교,다큐멘터리,"김한민, 오타니 료헤이, 이해영",[국내] 12세 관람가
2,어벤져스: 엔드게임,2019.4.24,1.221827e+11,"13,934,592","2,835","242,001",미국,월트디즈니컴퍼니코리아 유한책임회사,안소니 루소,액션,"로버트 다우니 주니어(토니 스타크 / 아이언맨), 크리스 에반스(스티브 로저스 / ...",[국내] 12세 관람가
3,베테랑,2015.8.5,1.051682e+11,"13,414,009","1,115","199,231",한국,(주)씨제이이엔엠,류승완,액션,"황정민(서도철), 유아인(조태오), 유해진(최상무)",[국내] 15세 관람가
4,신과함께-인과 연,2018.8.1,1.026661e+11,"12,274,996","2,235","179,981",한국,롯데컬처웍스(주)롯데엔터테인먼트,김용화,판타지,"하정우(강림), 주지훈(해원맥), 김향기(덕춘)",[국내] 12세 관람가
...,...,...,...,...,...,...,...,...,...,...,...,...
1042,더 프린세스 : 도둑맞은 공주,2020.8.12,4.574497e+08,"55,828",423,"4,192",우크라이나,(주)팝엔터테인먼트,올레 말라므,애니메이션,"사문영(밀라 목소리), 남도형(루슬란 목소리), 서반석(레스터 목소리)",[국내] 전체 관람가
1043,프리키 데스데이,2020.11.25,4.512257e+08,"51,812",631,"12,882",미국,유니버설픽쳐스인터내셔널 코리아(유),크리스토퍼 랜던,코미디,"빈스 본, 캐서린 뉴튼",[국내] 15세 관람가
1044,날씨의 아이,2019.10.30,4.449562e+08,"58,968",194,"5,921",일본,"메가박스중앙(주)플러스엠,(주)미디어캐슬,워터홀컴퍼니(주)",신카이 마코토,애니메이션,"심규혁(호다카), 김유림(히나), 최한(스가)",[국내] 15세 관람가
1045,눈의 여왕4,2019.12.24,4.446496e+08,"67,159",271,"2,239",러시아,(주)넥스트엔터테인먼트월드(NEW),로버트 렌스,애니메이션,"양정화(눈의 여왕 한국어 목소리), 박지윤(겔다 한국어 목소리), 임채헌(헤럴드 한...",[국내] 전체 관람가
